In [12]:
from langchain_aws import ChatBedrock
import os

claude_3_7_bedrock = ChatBedrock(
    model_id="arn:aws:bedrock:us-east-1:050451404360:inference-profile/us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    provider="anthropic",
    model_kwargs={"max_tokens": 20000,
    # "thinking": {"type": "enabled", "budget_tokens": 1024}
    },
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
)

claude_3_7_bedrock_thinking = ChatBedrock(
    model_id="arn:aws:bedrock:us-east-1:050451404360:inference-profile/us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    provider="anthropic",
    model_kwargs={"max_tokens": 20000,
    "thinking": {"type": "enabled", "budget_tokens": 1024}
    },
)


In [13]:
from langgraph.graph import END, StateGraph
from typing import Literal, TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

## GSheets

In [14]:
# login using service account
from google.oauth2 import service_account
import pandas as pd
import gspread
from gspread_pandas import Spread

# Configurar credenciais
credentials = service_account.Credentials.from_service_account_file(
    ".secrets/service-account-admin.json",
    scopes=['https://www.googleapis.com/auth/spreadsheets', 
            'https://www.googleapis.com/auth/drive']
)

# Conectar ao Google Sheets
gc = gspread.authorize(credentials)

# Extrair o ID da planilha da URL
sheet_id = "11I9QFSMFn7UBfV0wz0-hAYgWtIKytTVnWA9pjquwgdk"

# Abrir a planilha e obter a primeira aba
sheet = gc.open_by_key(sheet_id).sheet1

# Obter todos os dados e converter para DataFrame
data = sheet.get_all_values()
headers = data[0]
df = pd.DataFrame(data[1:], columns=headers)

# Exibir as primeiras linhas do DataFrame
df.head(3)

,name,Opiniao Pedro,focus,hard_commited,industry_agnostic,leader?,investment_geography,vc_quality_perception,observations,investment_range,...,Opiniao Cache,investor_profile,preferred_industry,preffered_industry_enriched,meeting_frequency,intros_made,intros_received,description,domains,entry_id
0,Bridge Latam,,,,TRUE,Leader and Follower,Latam,3.0,"Agnostic, low bar for investments",[< USD 1mn],...,,VC,[],"Business/Productivity Software, Financial Soft...",[Bimonthly],14.0,16,Bridge Partners is an early-stage investment f...,bridgelat.com,007037bc-f043-424c-a9ef-0d7394eeed90
1,Latitud,,,,,Pure-follower,Latam,4.0,Top-tier pre-seed dealflow,"[< USD 1mn, USD 1-5mn]",...,,VC,[],"Fintech, vertical SaaS, Healthtech, Marketplac...",[Monthly],13.0,13.0,Latitud is the operating system for Latin Amer...,latitud.com,009e1d00-ddaa-4e35-8113-3b775dfe46c7
2,Andreessen Horowitz,,,,,Leader and Follower,Global,5.0,Focusing more on Series A+,"[USD 1-5mn, USD 5-10mn, USD 10-20mn, > USD 20mn]",...,,VC,[],"(Global: AI, Bio + Healthcare, Consumer, Crypt...",[Monthly],14.0,3.0,Andreessen Horowitz is a venture capital firm ...,a16z.com,082a298f-a986-4686-becb-9905d63a806b


## Gdocs

In [15]:
# Importar bibliotecas necessárias para Google Docs
from googleapiclient.discovery import build
from google.oauth2 import service_account

# Configurar credenciais para Google Docs (usando as mesmas credenciais do Sheets)
SCOPES = ['https://www.googleapis.com/auth/documents.readonly']
creds = service_account.Credentials.from_service_account_file(
    '.secrets/service-account-admin.json',
    scopes=SCOPES
)

# Criar serviço do Google Docs
docs_service = build('docs', 'v1', credentials=creds)

# Função para obter conteúdo de um documento do Google Docs
def get_gdoc_content(doc_id):
    """
    Obtém o conteúdo de um documento do Google Docs pelo ID.
    
    Args:
        doc_id (str): ID do documento do Google Docs
        
    Returns:
        dict: Conteúdo do documento
    """
    try:
        # Fazer a requisição para obter o documento
        document = docs_service.documents().get(documentId=doc_id).execute()
        return document
    except Exception as e:
        print(f"Erro ao acessar o documento: {e}")
        return None


doc_ids = ["1WJDSO_uyTJ16DdkDOhgAUww4bMKuUa9fXuTh0HdkOew", "1F8jqD4pFq3_17NN75HZbzd2G2vrIqWaZqR1MOB5Fa6M", "182Qj0a8ZIXjS0_plWZdRY-aGY-X8qqpiIAUYex8xrd0"]
for doc_id in doc_ids:
    doc_content = get_gdoc_content(doc_id)
    if doc_content:
        print(f"Título do documento: {doc_content.get('title')}")


Título do documento: [AI MATERIAL] Definition of Lead and Follow
Título do documento: [AI MATERIAL] Geographic Focus
Título do documento: [AI MATERIAL] Preffered Industry


In [41]:
# Extrair texto do documento
doc_plaintext = ""
for item in doc_content.get('body').get('content'):
    if 'paragraph' in item:
        for element in item.get('paragraph').get('elements', []):
            if 'textRun' in element:
                doc_plaintext += element.get('textRun').get('content', '')

doc_plaintext

'Analysis of Venture Capital Investment Preferences in Latin America\n1. Executive Summary\nThis report analyzes the preferred investment industries of a selected group of venture capital firms with a significant presence or interest in the Latin American market. The analysis reveals a strong concentration of investment in the technology sector, particularly within software-as-a-service (SaaS), financial technology (Fintech), and e-commerce enablers. While many firms exhibit a broad interest across various technology verticals, some demonstrate specialization in areas such as agritech, edtech, and climate technology. The findings highlight the dynamism and increasing maturity of the Latin American venture capital ecosystem, attracting both regional and global investors who are keen to capitalize on the region\'s burgeoning entrepreneurial landscape. Understanding these investment preferences is crucial for entrepreneurs seeking funding and for investors looking to identify key opportun

## AI

In [17]:
inputs = {
    "company": "Brendi",
    "description_company": "Brendi is a company that creates AI agents to sell food in Brazilian restaurants via delivery. They are going to be the next ifood",
    "description_person": "Daniel is the CEO of Brendi. he studied at ITA, is very young and energetic",
    "round": {"size": "10M USD", "Funding": "Series A"},
    "round_commitment": "2M USD",
    "leader_or_follower": "leader",
    "industry": "AI Solutions, Food Delivery, Restaurant Management, AI Agents, Embedded Finance",
    "fund_closeness": "100%",
    "observations": "We are sure this deal is very hot, so we want the top funds with us in this one, but they have to fit"
}

parameters = {
    "batch_size": 10,
    "surviving_percentage": 0.5,
}


In [18]:
# First Filter
if inputs["leader_or_follower"] == "leader":
    # get only leader? that contains the word Leader (case insensitive)
    df = df[df["leader?"].str.lower().str.contains("leader")]
else:
    # get only follower? that contains the word Follower (case insensitive)
    df = df[df["leader?"].str.lower().str.contains("follower")]


In [19]:
from pydantic import BaseModel
class FundScore(TypedDict):
    fund_name: str
    score: float
    reason: str

class FundScoreList(BaseModel):
    scores: List[FundScore]

class AgentState(TypedDict):
    df: pd.DataFrame
    df_batches: list[pd.DataFrame]
    fund_scores: list[FundScore]
    raw_scores: List[FundScore]  # Armazenar pontuações não normalizadas
    second_raw_scores:List[FundScore]
    second_scores: List[FundScore]


BATCH_SIZE = 10
SURVIVING_PERCENTAGE = 0.5

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class FundScore(BaseModel):
    fund_name: str = Field(description="Fund Name")
    score: float = Field(description="Gross Score based on the sum of the criteria")
    reason: str = Field(description="Detailed reason for the score separated by criteria")

class FundScoreList(BaseModel):
    scores: List[FundScore]
    
class AgentState(TypedDict):
    df: pd.DataFrame
    fund_scores: List[FundScore]
    raw_scores: List[FundScore]  # Armazenar pontuações não normalizadas

def batch_splitter(df: pd.DataFrame) -> list[pd.DataFrame]:
    batch_size = BATCH_SIZE
    return [df[i:i+batch_size] for i in range(0, len(df), batch_size)]

def score_fund(state: AgentState) -> AgentState:
    system_prompt = """
    You are a fund score agent.
    You are given a table of funds, user inputs, and you need to score them based on the following criteria:

    - Meeting Frequency (the more frequent, the closer the fund is to us) | monthly = 10, bimonthly = 6, quarterly = 3, yearly = 0
    - VC quality (the best quality means the fund takes the best deals. A bad deal shouldn't go to a good fund. A bad deal should go to a bad fund) | 0-5 points
    - investment range (The investment range should be compatible with the round size - round commitment. If we want a leader, it should be at least half of the round size (then its a 5). If we want a follower, it should be smaller than half of the round size | 0-5 points
    - investment_geography (the fund's investment geography should be compatible with the user's investment geography) | -5 to 5 points. If the geography is a perfect match, the score is 5. If the geography is not a perfect match, the score is 3. If incompatible, the score is -5
    - funding_rounds_1st_check (the first check round should be compatible with the round type) | 0-5 points
    - observations (depends on the fund. If empty, score it 0 points) | -5 to 5 points
    - preffered_industry (the fund's preferred industry should be compatible with the company's industry. If there is just one intersection, the score is around 5. If there is a near perfect fit, the score is 10) | 0-10 points
    - industry_agnostic (if the fund is industry agnostic, it means the fund is not biased towards a specific industry) | afffects preffered_industry score
    - Description (the description should be compatible with the company's description) | 0-3 points

    {previous_scores_guidance}
    """

    human_prompt = """
    Here is the table of funds:
    {df}

    Here is the user inputs:
    {inputs}
    """

    batches = batch_splitter(state["df"])
    
    # Inicializar lista para armazenar scores não normalizados
    if "raw_scores" not in state:
        state["raw_scores"] = []

    for i, batch in enumerate(batches):
        # Preparar orientação baseada em pontuações anteriores
        previous_scores_guidance = ""
        if state["raw_scores"]:
            # Criar exemplos de pontuações anteriores para manter consistência
            examples = [(s.fund_name, s.score) for s in state["raw_scores"][:5]]
            previous_scores_guidance = f""" 
            IMPORTANT: Keep consistency with the scores already assigned to other funds.
            Examples of previous scores: {examples}
            Remember that the total score must be in an approximate scale with the scores already assigned.
            """
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt.format(previous_scores_guidance=previous_scores_guidance)),
            ("human", human_prompt)
        ])

        structured_llm = claude_3_7_bedrock.with_structured_output(FundScoreList)
        chain = prompt | structured_llm

        # Converter batch para string
        batch_str = batch.to_string()
        
        # Invocar o modelo
        fund_scores = chain.invoke({"df": batch_str, "inputs": inputs})
        
        # Guardar pontuações brutas
        state["raw_scores"].extend(fund_scores.scores)
        
        print(f"Processado lote {i+1}/{len(batches)}")

    # Normalizar todas as pontuações no final
    if state["raw_scores"]:
        min_score = min(score.score for score in state["raw_scores"])
        max_score = max(score.score for score in state["raw_scores"])
        
        if max_score > min_score:  # evitar divisão por zero
            state["fund_scores"] = [
                FundScore(
                    fund_name=score.fund_name,
                    score=100 * (score.score - min_score) / (max_score - min_score),
                    reason=score.reason
                ) for score in state["raw_scores"]
            ]
        else:
            # Caso todas as pontuações sejam iguais
            state["fund_scores"] = [
                FundScore(
                    fund_name=score.fund_name,
                    score=50.0,  # valor médio arbitrário
                    reason=score.reason
                ) for score in state["raw_scores"]
            ]

    return state

initial_state = AgentState(
    df=df,
    fund_scores=[],
    raw_scores=[]
)

result = score_fund(initial_state)

Processado lote 1/4
Processado lote 2/4
Processado lote 3/4
Processado lote 4/4


In [32]:
sorted_results = sorted(result["fund_scores"], key=lambda x: x.score, reverse=True)
sorted_results

[FundScore(fund_name='Monashees', score=100.0, reason="Meeting Frequency: 10 (monthly meetings)\nVC quality: 5 (top-tier fund)\nInvestment range: 4 (offers USD 5-10mn range suitable for Series A round of 10M)\nInvestment geography: 5 (Brazil-focused which perfectly matches the company's location)\nFunding rounds: 5 (invests 'Regardless' of round type, including Series A)\nObservations: 3 (top-tier fund matching the requirement for quality partners)\nPreferred industry: 5 (explicitly mentions marketplaces, e-commerce, and tech companies with high potential scale which aligns well with Brendi's AI-powered food delivery business)"),
 FundScore(fund_name='Kaszek', score=96.66666666666667, reason="Meeting Frequency: 10 (monthly meetings show high engagement)\nVC quality: 5 (top-tier fund with excellent reputation)\nInvestment range: 4 (offers USD 5-10mn range which fits Series A with 10M round size, though as a leader they might need more commitment)\nInvestment geography: 5 (focused on Lat

In [33]:
import math
to_keep = parameters["surviving_percentage"]*len(result["fund_scores"])
to_keep = math.ceil(to_keep)
to_keep

20

In [38]:
kept_funds = sorted_results[:to_keep]
kept_funds

[FundScore(fund_name='Monashees', score=100.0, reason="Meeting Frequency: 10 (monthly meetings)\nVC quality: 5 (top-tier fund)\nInvestment range: 4 (offers USD 5-10mn range suitable for Series A round of 10M)\nInvestment geography: 5 (Brazil-focused which perfectly matches the company's location)\nFunding rounds: 5 (invests 'Regardless' of round type, including Series A)\nObservations: 3 (top-tier fund matching the requirement for quality partners)\nPreferred industry: 5 (explicitly mentions marketplaces, e-commerce, and tech companies with high potential scale which aligns well with Brendi's AI-powered food delivery business)"),
 FundScore(fund_name='Kaszek', score=96.66666666666667, reason="Meeting Frequency: 10 (monthly meetings show high engagement)\nVC quality: 5 (top-tier fund with excellent reputation)\nInvestment range: 4 (offers USD 5-10mn range which fits Series A with 10M round size, though as a leader they might need more commitment)\nInvestment geography: 5 (focused on Lat

In [42]:
doc_plaintext

'Analysis of Venture Capital Investment Preferences in Latin America\n1. Executive Summary\nThis report analyzes the preferred investment industries of a selected group of venture capital firms with a significant presence or interest in the Latin American market. The analysis reveals a strong concentration of investment in the technology sector, particularly within software-as-a-service (SaaS), financial technology (Fintech), and e-commerce enablers. While many firms exhibit a broad interest across various technology verticals, some demonstrate specialization in areas such as agritech, edtech, and climate technology. The findings highlight the dynamism and increasing maturity of the Latin American venture capital ecosystem, attracting both regional and global investors who are keen to capitalize on the region\'s burgeoning entrepreneurial landscape. Understanding these investment preferences is crucial for entrepreneurs seeking funding and for investors looking to identify key opportun

In [44]:
fund_names = []
for fund in kept_funds:
    fund_names.append(fund.fund_name)
fund_names

['Monashees',
 'Kaszek',
 'Atlantico',
 'Valor Capital Group',
 'big_bets',
 'Alexia Ventures',
 'ONEVC',
 'Upload',
 'Headline',
 'Astella',
 'DST Global',
 'Andreessen Horowitz',
 'SaaSholic',
 'NXTP Ventures',
 'Zenda',
 'Quona Capital',
 'Dalus Capital',
 'Canary',
 'J.P. Morgan & Co.',
 'Fen Ventures']

In [46]:
df_first_filter = df[df["name"].isin(fund_names)]
df_first_filter

,name,Opiniao Pedro,focus,hard_commited,industry_agnostic,leader?,investment_geography,vc_quality_perception,observations,investment_range,...,Opiniao Cache,investor_profile,preferred_industry,preffered_industry_enriched,meeting_frequency,intros_made,intros_received,description,domains,entry_id
2,Andreessen Horowitz,,,,,Leader and Follower,Global,5.0,Focusing more on Series A+,"[USD 1-5mn, USD 5-10mn, USD 10-20mn, > USD 20mn]",...,,VC,[],"(Global: AI, Bio + Healthcare, Consumer, Crypt...",[Monthly],14.0,3.0,Andreessen Horowitz is a venture capital firm ...,a16z.com,082a298f-a986-4686-becb-9905d63a806b
3,big_bets,,,,,Leader and Follower,Global,4.0,Brazil and Global thesis,"[< USD 1mn, USD 1-5mn]",...,,VC,[],"Software (IDtech, AI, SaaS, Fintech, Edtech, G...",[Monthly],18.0,44.0,big_bets is a venture capital and private equi...,bigbets.com.br,0920744a-6cdf-4992-9ecd-f370cb240be6
4,NXTP Ventures,,,,TRUE,Leader,Latam,4.0,"B2B focused, Fintech, Marketplace, Ecommerce.","[USD 1-5mn, < USD 1mn]",...,,VC,[],"Cloud & SaaS, E-commerce Enablers, Fintech, B2...",[Monthly],17.0,6.0,NXTP Ventures is a pioneering early stage vent...,nxtp.vc,099a7873-7123-4eb9-8397-6e629e61953c
5,Alexia Ventures,,,,,Leader and Follower,Latam,4.0,SaaS and AI businesses,"[< USD 1mn, USD 1-5mn, USD 5-10mn, USD 10-20mn]",...,,VC,[],"Software Platforms, Data/AI, Blockchain/Crypto...",[Monthly],29.0,13.0,Alexia Ventures is a venture capital firm that...,alexia.vc,0b184e1c-3f6e-4c24-b0b7-5d39ac18a8a9
12,SaaSholic,,,,,Leader and Follower,Latam,3.0,"B2B businesses, low valuations","[< USD 1mn, USD 1-5mn]",...,,VC,[],"SaaS, other tech-related (Future of Work, Mobi...",[Monthly],15.0,8.0,SaaSholic is an early stage VC firm investing ...,saasholic.com,1a89537c-1147-43d2-9e5f-046e10da584f
14,Dalus Capital,,,,,Leader and Follower,Latam,3.0,"Focuses on technology, healthcare, climate, an...","[< USD 1mn, USD 1-5mn]",...,,VC,"[Healthtech, Mobility and Supply Chain, Climate]","Technology, Climate Innovation, Business Produ...",[Bimonthly],7.0,0.0,Dalus Capital is an international venture capi...,daluscapital.com,1dc017fd-9865-475e-9a6f-f16068940de8
18,Zenda,,,,,Leader and Follower,Latam,3.0,"Fintech, Logistics & Mobility, Healthtech, AI ...",[< USD 1mn],...,,VC,"[Fintech, Logistics & Mobility, Healthtech, AI...","Healthcare, Financial Services, Business Produ...",[Bimonthly],18.0,3,Zenda is a thesis-driven tech investment firm ...,zenda.vc,37b17e4a-4825-4319-b602-645c3351bbdb
19,ONEVC,,,,,Leader and Follower,Latam,5.0,"Solo players, rising top-tier fund","[< USD 1mn, USD 1-5mn, USD 5-10mn]",...,,VC,[],"Fintech, SaaS, Logistics, Proptech, Biotechnol...",[Bimonthly],23.0,6.0,ONEVC is a venture capital firm based in Latam...,onevc.vc,3989b2b9-cb06-4ee5-938f-6a119dd8c65d
20,Fen Ventures,,,,,Leader and Follower,Latam,3.0,,[],...,,VC,[],"Digital Consumer (Home, Entertainment, Healthc...",[Bimonthly],1.0,1.0,Fen Ventures: Early stage VC partnering with t...,fenventures.com,41340153-90bc-4f59-8c3f-73ec769ba631
32,Headline,,,,FALSE,Leader,Global,3.0,"Out of the mainstream, PE-like deals","[USD 1-5mn, USD 5-10mn]",...,,VC,[],Growth-stage Technology,[Bimonthly],13.0,1.0,Headline (formerly e.ventures) is a Venture Ca...,headline.com,7d62afe2-3576-4410-8770-17dbc24f37b0


In [54]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash"
)

In [55]:
def score_fund(state: AgentState) -> AgentState:
    system_prompt = """
You are a fund score agent.
You are given a list of funds, a document describing them and some criteria to score

Score from 1 to 10 to each fund based on how you judge the fund would like to invest in the company

    {previous_scores_guidance}
    """

    human_prompt = """
    Here is the table of funds:
    {df}

    Here is the document describing the funds:
    {doc_plaintext}

    Here is the user inputs:
    {inputs}
    """

    batches = batch_splitter(state["df"])
    
    # Inicializar lista para armazenar scores não normalizados
    if "second_raw_scores_list" not in state:
        state["second_raw_scores_list"] = []

    for i, batch in enumerate(batches):
        # Adicionar espera de 30 segundos entre chamadas para evitar muitas tentativas
        if i > 0:
            import time
            time.sleep(30)

        useful_cols = ["name", "description", "preffered_industry_enriched", "preferred_industry", "observations", "Opiniao Pedro", "Opiniao Gus", "Opiniao Cache"]
        batch = batch[useful_cols]

        # Preparar orientação baseada em pontuações anteriores
        previous_scores_guidance = ""
        if state["second_raw_scores_list"]:
            # Criar exemplos de pontuações anteriores para manter consistência
            examples = [(s.fund_name, s.score) for s in state["second_raw_scores_list"][:5]]
            previous_scores_guidance = f""" 
            IMPORTANT: Keep consistency with the scores already assigned to other funds.
            Examples of previous scores: {examples}
            Remember that the total score must be in an approximate scale with the scores already assigned.
            """
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt.format(previous_scores_guidance=previous_scores_guidance)),
            ("human", human_prompt)
        ])

        structured_llm = claude_3_7_bedrock.with_structured_output(FundScoreList)
        chain = prompt | structured_llm

        # Converter batch para string
        batch_str = batch.to_string()
        
        # Invocar o modelo
        second_scores = chain.invoke({"df": batch_str, "inputs": inputs, "doc_plaintext": doc_plaintext})
        
        # Guardar pontuações brutas
        state["second_raw_scores_list"].extend(second_scores.scores)
        
        print(f"Processado lote {i+1}/{len(batches)}")

    # Normalizar todas as pontuações no final
    if state["second_raw_scores_list"]:
        min_score = min(score.score for score in state["second_raw_scores_list"])
        max_score = max(score.score for score in state["second_raw_scores_list"])
        
        if max_score > min_score:  # evitar divisão por zero
            state["second_scores"] = [
                FundScore(
                    fund_name=score.fund_name,
                    score=100 * (score.score - min_score) / (max_score - min_score),
                    reason=score.reason
                ) for score in state["second_raw_scores_list"]
            ]
        else:
            # Caso todas as pontuações sejam iguais
            state["second_scores"] = [
                FundScore(
                    fund_name=score.fund_name,
                    score=50.0,  # valor médio arbitrário
                    reason=score.reason
                ) for score in state["second_raw_scores_list"]
            ]

    return state


initial_state = AgentState(
    df=df,
    fund_scores=[],
    raw_scores=[],
    second_raw_scores_list=[],
    second_scores=[]
)

result = score_fund(initial_state)


Processado lote 1/4


Error raised by bedrock service
Traceback (most recent call last):
  File "c:\Projetos\Norte\walter-intro-maker\venv\Lib\site-packages\langchain_aws\llms\bedrock.py", line 956, in _prepare_input_and_invoke
    response = self.client.invoke_model(**request_options)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Projetos\Norte\walter-intro-maker\venv\Lib\site-packages\botocore\client.py", line 570, in _api_call
    return self._make_api_call(operation_name, kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Projetos\Norte\walter-intro-maker\venv\Lib\site-packages\botocore\context.py", line 124, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Projetos\Norte\walter-intro-maker\venv\Lib\site-packages\botocore\client.py", line 1031, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.ThrottlingException: An error occurred (ThrottlingException) when calling the Invo

ThrottlingException: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many tokens, please wait before trying again.